In [1]:
# Run if working locally
%load_ext autoreload
%autoreload 2
%load_ext nb_black

<IPython.core.display.Javascript object>

In [3]:
import sqlite3
from sqlite3 import Error
import pickle
import os, sys
import config

config.root_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.insert(0, config.root_path)

from src.dataset.dataset import RawData
from src.dataset.wikisection_preprocessing import (
    tokenize,
    clean_sentence,
    preprocess_text_segmentation,
    format_data_for_db_insertion,
)
from src.dataset.utils import truncate_by_token
from db.dbv2 import Table, AugmentedTable, TrainTestTable
import pprint


from utils.metrics import windowdiff, pk

from src.bertkeywords.src.similarities import Embedding, Similarities
from src.bertkeywords.src.keywords import Keywords
from src.encoders.coherence_v2 import Coherence
from src.dataset.utils import flatten, dedupe_list, truncate_string

<IPython.core.display.Javascript object>

In [4]:
dataset_type = "city"
table = Table(dataset_type)
augmented_table = AugmentedTable(dataset_type)
train_test_table = TrainTestTable(dataset_type)

<IPython.core.display.Javascript object>

In [5]:
data = table.get_all()

text_data = [x[1] for x in data]
text_labels = [x[2] for x in data]

<IPython.core.display.Javascript object>

In [6]:
all_segments = table.get_all_segments()

segments = [[y[1] for y in x] for x in all_segments]
segments_labels = [[1 if i == 0 else 0 for i, y in enumerate(x)] for x in all_segments]

<IPython.core.display.Javascript object>

In [7]:
flattened_segments = flatten(segments)
flattened_labels = flatten(segments_labels)
len(flattened_segments), len(flattened_labels)

(92833, 92833)

<IPython.core.display.Javascript object>

## KeyBERT

In [8]:
testing_data = flattened_segments[:5]

<IPython.core.display.Javascript object>

In [9]:
testing_data

['In spite of appearances, both the Basque form Donostia and the Spanish form San Sebastián have the same meaning of Saint Sebastian. The dona/done/doni element in Basque place-names signifies "saint" and is derived from Latin domine; the second part of Donostia contains a shortened form of the saint\'s name. There are two hypotheses regarding the evolution of the Basque name: one says it was *Done Sebastiáne > Donasa(b)astiai > Donasastia > Donastia > Donostia, the other one says it was *Done Sebastiane > *Done Sebastiae > *Done Sebastie > *Donesebastia > *Donasastia > *Donastia > Donostia.\n',
 "The city is in the north of the Basque Autonomous Community, on the southern coast of the Bay of Biscay. San Sebastián's picturesque shoreline makes it a popular beach resort. The seaside environment is enhanced by hilly surroundings that are easily accessible, i.e., Urgull (at the heart of the city by the seashore), romantic Mount Ulia extending east to Pasaia, Mount Adarra rising proud far 

<IPython.core.display.Javascript object>

In [33]:
from src.bertkeywords.src.keybert import KeyBERT

<IPython.core.display.Javascript object>

In [28]:
kw_model = KeyBERT(model="bert-base-uncased")

doc_embeddings, word_embeddings = kw_model.extract_embeddings(
    testing_data[0], keyphrase_ngram_range=(1, 1)
)

keywords = kw_model.extract_keywords(
    testing_data[0],
    doc_embeddings=doc_embeddings,
    word_embeddings=word_embeddings,
    keyphrase_ngram_range=(1, 1),
)


No sentence-transformers model found with name /Users/amitmaraj/.cache/torch/sentence_transformers/bert-base-uncased. Creating a new one with MEAN pooling.
Some weights of the model checkpoint at /Users/amitmaraj/.cache/torch/sentence_transformers/bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassificatio

<IPython.core.display.Javascript object>

In [24]:
len(word_embeddings)

36

<IPython.core.display.Javascript object>

In [31]:
len(keywords[0][2])

768

<IPython.core.display.Javascript object>